In [110]:
import pandas as pd
import numpy as np

In [111]:
donnees_mes = pd.read_excel("../data/donnees_mesures_PM2_5.xlsx")
donnees_env = pd.read_excel("../data/donnees_environnement_capteurs.xlsx")
donnees_mes

,Date,c1,c3,c5,c7,c8,c10,c11,c13,c14,...,c31,c87,c50,c51,c64,c73,c74,c75,c76,c90
0,2023-01-01 00:00:00,4.678,4.496000,4.505000,4.607,2.790,4.222,5.668,5.769,4.416,...,5.810,7.3360,7.312,4.912,27.0110,4.737,3.080,4.009,7.409,3.918
1,2023-01-01 00:15:00,4.439,NaN,NaN,4.330,3.167,4.077,4.423,6.315,5.549,...,5.732,8.2400,5.480,5.259,30.1920,4.764,3.168,3.999,7.021,4.132
2,2023-01-01 00:30:00,4.108,NaN,NaN,4.547,3.081,3.871,4.744,5.356,4.775,...,5.611,NaN,7.717,4.599,19.4200,4.922,3.194,4.231,NaN,4.079
3,2023-01-01 00:45:00,4.713,4.339667,3.982333,4.688,NaN,3.984,5.499,5.329,4.961,...,5.717,8.4075,4.923,4.927,31.3880,4.721,3.517,4.252,6.090,3.876
4,2023-01-01 01:00:00,4.172,3.716000,4.088000,NaN,3.556,3.960,4.833,5.613,4.462,...,5.858,8.7640,4.211,4.539,25.8640,4.651,3.121,4.141,6.178,3.590
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25779,2023-09-26 12:45:00,1.768,1.892000,1.529000,2.483,1.824,NaN,2.196,NaN,2.567,...,1.094,2.1650,1.937,3.240,2.2635,5.485,NaN,4.470,2.053,0.762
25780,2023-09-26 13:00:00,1.857,1.792000,1.538000,2.548,2.405,NaN,2.262,NaN,2.857,...,1.145,2.1920,2.532,2.328,2.2690,2.182,NaN,4.658,2.385,0.639
25781,2023-09-26 13:15:00,1.939,1.871000,1.953000,2.207,1.761,NaN,2.056,NaN,2.781,...,1.520,2.2820,3.054,2.312,2.6060,2.625,NaN,3.606,2.855,0.769
25782,2023-09-26 13:30:00,1.959,NaN,2.028000,2.415,1.844,NaN,2.498,NaN,2.914,...,1.245,2.6810,2.054,3.560,2.1560,2.835,NaN,2.318,3.037,1.560


In [112]:
date = donnees_mes['Date']
donnees_mes = donnees_mes.drop(donnees_mes.columns[0], axis=1)

# Remplacement des virgules par des points et conversion en float pour toutes les colonnes
for nom_colonne in donnees_mes.columns:
    donnees_mes[nom_colonne] = donnees_mes[nom_colonne].replace(',', '.').astype(float)

In [113]:
# [['c7', 37.93437790878064], ['c26', 46.95547626434998], ['c27', 41.79336022339435]] with NaN before aberrant

In [ ]:
#Suppression des valeurs aberrantes

donnees_mes.hist()
    
for colonne in donnees_mes.columns:
        if pd.api.types.is_numeric_dtype(donnees_mes[colonne]):
            Q1 = donnees_mes[colonne].quantile(0.25)
            Q3 = donnees_mes[colonne].quantile(0.75)
            IQR = Q3 - Q1
            
            # Détection des valeurs aberrantes
            outliers_mask = (donnees_mes[colonne] < (Q1 - 1.5 * IQR)) | (donnees_mes[colonne] > (Q3 + 1.5 * IQR))
            
            # Remplacement des valeurs aberrantes par NaN
            donnees_mes.loc[outliers_mask, colonne] = np.nan
        


In [ ]:
# Supression des colonnes aberrantes et remplissage des valeurs nulles

# Compter le nombre de valeurs manquantes par colonne
nombre_na_par_colonne = donnees_mes.isna().sum()

# Supprimer les colonnes avec un pourcentage de valeurs manquantes supérieur à 33%
colonnes_del = []
for nom_colonne, nombre_na in nombre_na_par_colonne.items():
    if nombre_na / len(donnees_mes) > 0.33:
        colonnes_del.append([nom_colonne,nombre_na*100/len(donnees_mes)])
        donnees_mes = donnees_mes.drop(nom_colonne, axis=1)
        

# Remplir les valeurs manquantes restantes avec la médiane de chaque colonne
donnees_mes = donnees_mes.fillna(donnees_mes.median())

print(colonnes_del)
donnees_mes.hist()

In [ ]:
from sklearn.preprocessing import StandardScaler

# Normalisation des données
nom_colonnes = list(donnees_mes.columns)
scaler = StandardScaler()
X = donnees_mes.to_numpy()
X = scaler.fit_transform(X)
mes_normees = pd.DataFrame(X)
mes_normees


In [ ]:
mes_normees.columns=nom_colonnes
mes_normees.insert(0, 'Date', date)


In [ ]:
mes_normees.to_csv("../preprocessing/donnees_mesures_PM2_5.csv",index=False,sep=';')